In [61]:
import praw
import random
from praw.models import MoreComments
from bs4 import BeautifulSoup
from markdown import markdown
from google.cloud import texttospeech
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from moviepy.editor import *
from mutagen.mp3 import MP3
import time

In [38]:
reddit = praw.Reddit(
    client_id="zkUPC3t9gt5z0k9zIq1_5Q",
    client_secret="g02-Fk_OK98vTkFK6pgz7nnI1kyA0w",
    password="password",
    user_agent="testscript",
    username="amaz_ingPerson",
)
print(reddit.user.me())

amaz_ingPerson


In [39]:
links = []
for submission in reddit.subreddit("AskReddit").hot(limit=25):
    links.append(submission.url)

print(links)

['https://www.reddit.com/r/AskReddit/comments/w3gg0w/whats_a_funny_memory_you_have_that_if_you_told/', 'https://www.reddit.com/r/AskReddit/comments/w35ocq/whats_a_fun_fact_that_nobody_asked_for/', 'https://www.reddit.com/r/AskReddit/comments/w330mh/what_can_someone_put_on_their_car_that_makes_you/', 'https://www.reddit.com/r/AskReddit/comments/w3e1qc/what_would_be_the_most_terrifying_message_we_can/', 'https://www.reddit.com/r/AskReddit/comments/w36uts/youve_just_walked_in_on_your_wife_cheating_on_you/', 'https://www.reddit.com/r/AskReddit/comments/w3mair/what_is_a_wholesome_animal_fact_you_know/', 'https://www.reddit.com/r/AskReddit/comments/w3aerk/what_is_moral_yet_illegal/', 'https://www.reddit.com/r/AskReddit/comments/w311h6/in_your_humble_opinion_what_tv_series_has_the/', 'https://www.reddit.com/r/AskReddit/comments/w3ljgp/john_wick_has_a_guy_for_his_guns_a_guy_for_his/', 'https://www.reddit.com/r/AskReddit/comments/w3gpmr/what_is_something_we_commonly_accept_as_true_in/', 'https:

In [49]:
submission_url = random.choice(links)
submission_id = submission_url.replace('https://www.reddit.com/r/AskReddit/comments/', "")
submission_id = submission_id[:6]
submission = reddit.submission(submission_id)

print(submission.title)

People who talk on speakerphone in public, how dare you and who do you think you are?


In [52]:
comments = []
reddit_comments = []
for comment in submission.comments:
    if isinstance(comment, MoreComments):
        continue
     
    html = markdown(comment.body)
    text = ''.join(BeautifulSoup(html).findAll(text=True))
    
    if len(text) < 200:
        comments.append(text)
        reddit_comments.append(comment)

random_number = random.randint(0, len(comments))

comment = comments[random_number]
comment_permalink = reddit_comments[random_number].permalink
comment_url = 'https://reddit.com' + comment_permalink + '?utm_source=share&utm_medium=web2x&context=3'
print(comment)
print(comment_url)

I will say that I've done this but I walk away from a crowd before hand. Reason being my old phone only works for calls if it was on speaker 🤣
https://reddit.com/r/AskReddit/comments/w3tkos/people_who_talk_on_speakerphone_in_public_how/igy8w7n/?utm_source=share&utm_medium=web2x&context=3


In [53]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\helen\\Documents\\GitHub\\Youtube-Reddit-Video\\youtube-project-356203-9d9c9919829a.json"
client = texttospeech.TextToSpeechClient()
synthesis_input = texttospeech.SynthesisInput(text=comment)
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US",
    name="en-US-Wavenet-D",
    ssml_gender=texttospeech.SsmlVoiceGender.MALE
)
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)
response = client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)
with open("body_audio.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Comment text written to file "body_audio.mp3"')
    
synthesis_input = texttospeech.SynthesisInput(text=submission.title)
response = client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)
with open("title_audio.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Title text written to file "title_audio.mp3"')

Comment text written to file "body_audio.mp3"
Title text written to file "title_audio.mp3"


In [65]:
option = Options()
option.add_argument('--disable-notifications')
driver = webdriver.Chrome(executable_path= "C:\\Users\\helen\\Documents\\GitHub\\Youtube-Reddit-Video\\chromedriver.exe", chrome_options= option)

driver.get(comment_url)
title_element = driver.find_element(By.CLASS_NAME, "_3MC4c3Q_Y41YKtl1TcvyMt")
title_element.screenshot("title_screenshot.png")
time.sleep(1)
body_element = driver.find_element(By.CLASS_NAME, "_3sf33-9rVAO_v4y0pIW_CH")
body_element.screenshot("body_screenshot.png")
driver.quit()

In [66]:
title_audio = MP3("title_audio.mp3")
title_length = title_audio.info.length
if round(title_length + 1) > title_length + 1:
    title_length = round(title_length)
else:
    title_length = round(title_length + 1)
    
body_audio = MP3("body_audio.mp3")
body_length = body_audio.info.length
if round(body_length + 1) > body_length + 1:
    body_length = round(body_length)
else:
    body_length = round(body_length + 1)
    
print("Title.mp3 Length: " + str(title_length))
print("Body.mp3 Length: " + str(body_length))

Title.mp3 Length: 6
Body.mp3 Length: 10


In [71]:
random_point = random.randint(0, 2220)

video = VideoFileClip("background.mp4").subclip(random_point, random_point + title_length + body_length)

title_image = ImageClip("title_screenshot.png")
title_audio = AudioFileClip("title_audio.mp3")
title_image = title_image.set_duration(title_length)
title_image = title_image.set_audio(title_audio)

body_image = ImageClip("body_screenshot.png")
body_audio = AudioFileClip("body_audio.mp3")
body_image = body_image.set_duration(body_length)
body_image = body_image.set_audio(body_audio)

images = concatenate_videoclips([title_image, body_image], method="compose")

final = CompositeVideoClip([video, images.set_position("center")])
final.write_videofile("video.mp4")

Moviepy - Building video video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
